# Calendarización de trabajos (job scheduling)

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$

Los problemas de calendarización de trabajos (*job scheduling*) consisten en determinar el orden óptimo en el que deben ejecutarse ciertas tareas para optimizar un proceso productivo.

Suponer que se tiene dado un conjunto de $n$ trabajos de confección $J:= \tabulatedset{1, \ldots, n}$, los cuales  deben ser procesados secuencialmente en una máquina hiladora. La máquina contiene un carrete en el cual pueden ser cargados simultáneamente $B$ hilos de diferentes colores, seleccionados de un conjunto $H$ de hilos disponibles. Cada trabajo $j \in J$  requiere para su ejecución de un subconjunto $H_j \subset H$ de hilos. Para que $j$
pueda ser procesado, todos los hilos de $H_j$ deben estar cargados en el carrete. Si esto no ocurre, la máquina debe ser detenida para cargar los hilos que hagan falta y, de ser necesario, descargar hilos que no sean requeridos con la finalidad de liberar espacio en el carrete. Asumimos que $\card{H_j} \leq B, \forall j \in J$.

Se desea determinar el orden en el que los trabajos deben ser procesados para minimizar el número de paradas requeridas de la máquina. Para especificar el ordenamiento, se asignará a cada trabajo $j \in J$ un turno $t \in T$, con $T:= \tabulatedset{1, \ldots, n}$. 

Emplearemos las siguientes variables de decisión binarias: 

* $x_{jt}$, $j \in J$, $t \in T$, que indican si el trabajo $j$ es procesado en el turno $t$;
* $y_{ht}$, $h \in H$, $t \in T$, que determinan si el hilo $h \in H$ está cargado en la máquina durante el turno $t$; 
* $z_t$, $t \in T, \, t \geq 2$, que registran si es necesario parar la máquina entre los turnos $t-1$ y $t$.

Los valores de $z_t$ se determinan a partir de los valores de $y_{ht}$ como se indica a continuación. Notar que la expresión es $\card{y_{ht} - y_{h,t-1}}$ es igual a 1 si el hilo $h$ entra o sale del carrete de la máquina entre los turnos $t$ y $t-1$. Por lo tanto, definiendo:
$$
w_{ht} \in \{0, 1 \} \, : w_{ht} \geq \max\{ y_{ht} - y_{h,t-1}, y_{h,t-1} - y_{ht} \},
$$
y $z_{t} \in \{0, 1 \}$, $z_t \geq \frac{\sum_{h \in H} w_{ht}}{\card{H}}$, se tiene que $z_t = 1$ si es necesario detener la máquina entre los turnos $t-1$ y $t$ para cambiar algún hilo.

Empleando estas ideas, el problema de calendarización de trabajos descrito arriba puede ser formulado como el siguiente modelo de programación lineal entera.  

\begin{align*}
\min &\sum_{t \in T \setminus \{1\}} z_t\\ 
& \mbox{s.r.}\\
&\sum_{j \in J} x_{jt} = 1, \quad \forall t \in T, \\
&\sum_{t \in T} x_{jt} = 1, \quad \forall j \in J, \\
&\sum_{h \in H} y_{ht} \leq B, \quad \forall t \in T, \\
& x_{jt} \leq y_{ht}, \quad \forall h \in H_j, \, j \in J, \, t \in T,\\
& w_{ht} \geq y_{ht} - y_{h, t-1}, \quad, \forall h \in H, t \in T \setminus \{1\},\\
& w_{ht} \geq y_{h, t-1} - y_{ht}, \quad, \forall h \in H, t \in T \setminus \{1\},\\
& \sum_{h \in H} w_{ht} \leq \card{H} z_t, \forall t \in T \setminus \{1\},\\
& x_{jt}, y_{ht}, w_{ht}, z_t \in \{0, 1\}, \quad \forall j \in J, h \in H, t \in T.
\end{align*}


Vamos a implementar este modelo usando la interfaz Python de Gurobi.


Definimos primero los conjuntos $J$, $H$, $T$; y los parámetros $H_j$ y $B$:

In [22]:
from gurobipy import *
import random

# Conjuntos y parametros del modelo
# Familia HH de conjuntos de hilos requeridos por cada trabajo
#HH = tupledict({1 : {2, 3, 5},
#                2 : {1, 3, 4},
#                3 : {1, 4, 5},
#                4 : {1, 3},
#                5 : {1, 2, 4},
#                6 : {1, 4, 5},
#7 : {3, 5}})
HH = tupledict({})
for j in range(1,101):
    Hj = set()
    for h in range(3):
        Hj = Hj | set({random.randint(1,5)})
    HH[j] = Hj

print(HH)

# Cantidad de hilos que pueden estar en el carrete
B = 3

# ---- a partir de aqui, los parametros se calculan en base a los anteriores ----

# Conjunto de trabajos
J = HH.keys()

# Conjunto de turnos T y de turnos sin el primero
T = J
T2 = [i for i in T if i!=1]

# Conjunto de todos los hilos, calculado como la union de Hj para j in J
H = set()
for j in J:
    H = H | HH[j]
print(H)

{1: set([2, 3]), 2: set([1, 4, 5]), 3: set([1, 4]), 4: set([2, 3, 4]), 5: set([2, 4]), 6: set([3]), 7: set([1, 4, 5]), 8: set([1, 3]), 9: set([2, 3, 4]), 10: set([2, 4, 5]), 11: set([3, 4, 5]), 12: set([1, 4]), 13: set([2, 3, 4]), 14: set([2, 3]), 15: set([2, 3]), 16: set([1, 4]), 17: set([1, 5]), 18: set([3, 4]), 19: set([1, 3, 4]), 20: set([1, 3, 4]), 21: set([1, 2, 5]), 22: set([3, 5]), 23: set([1, 3, 5]), 24: set([3, 5]), 25: set([3, 4, 5]), 26: set([3, 4]), 27: set([1, 2]), 28: set([2, 3, 4]), 29: set([2, 4]), 30: set([3, 4, 5]), 31: set([1, 2]), 32: set([3, 4, 5]), 33: set([2, 3]), 34: set([2, 3, 4]), 35: set([4, 5]), 36: set([2, 3]), 37: set([1, 2, 4]), 38: set([1, 3]), 39: set([4, 5]), 40: set([4]), 41: set([2, 4, 5]), 42: set([2, 5]), 43: set([1]), 44: set([1, 2, 3]), 45: set([4, 5]), 46: set([2, 3]), 47: set([1, 3, 5]), 48: set([2, 3, 4]), 49: set([1, 5]), 50: set([3, 4, 5]), 51: set([1, 3]), 52: set([1, 2]), 53: set([3, 4, 5]), 54: set([2, 3, 5]), 55: set([1, 3]), 56: set([1

Definimos ahora el objeto modelo y las variables del modelo:

In [23]:
m = Model('job-scheduling')

# Asignacion turnos a trabajos
x = m.addVars(J, T, vtype = GRB.BINARY, name="x")

# Estado de hilos en el carrete
y = m.addVars(H, T, vtype = GRB.BINARY, name="y")

# Paradas de la maquina
z = m.addVars(T2, vtype = GRB.BINARY, name="z")

# Auxiliar: cambio de los hilos en el carrete
w = m.addVars(H, T2, vtype = GRB.BINARY, name="w")

Definimos la función objetivo:

In [24]:
# minimizar paradas de la maquina
m.setObjective(z.sum('*'), GRB.MINIMIZE)

Finalmente, implementamos las restricciones del modelo. El método `addConstrs` nos permite agregar una familia completa de restricciones al modelo, empleando un generador.

1. A cada trabajo $j$ se le asigna un turno único.

In [25]:
# A cada turno se le asigna exactamente un trabajo
m.addConstrs((x.sum('*', t) == 1 for t in T), name='turno_t') 

# A cada trabajo se le asigna exactamente un turno
m.addConstrs((x.sum(j, '*') == 1 for j in J), name='turno_j') 

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>,
 22: <gurobi.Constr *Awaiting Model Update*

2. En cada turno, la cantidad de hilos cargados en la máquina no supera la capacidad del carrete.

In [26]:
# Respetar capacidad del carrete
m.addConstrs((y.sum('*', t) <= B for t in T), name='carrete') 

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>,
 22: <gurobi.Constr *Awaiting Model Update*

3. Si el trabajo $j$ es procesado en el turno $t$, entonces todos los hilos requeridos para este trabajo deben estar cargados en el carrete en el turno $t$.

In [27]:
# Cargar todos los hilos requeridos para procesar un trabajo
m.addConstrs((x[j,t] <= y[h,t] for t in T for j in J for h in HH[j]), name='hilos') 

{(1, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9, 2): <gurobi.Constr *Awaiting Model Upd

4. Definición de las variables auxiliares:

In [28]:
# Definir w[h,t] en función de y[h,t] y y[h,t-1]
m.addConstrs((w[h,t] >= y[h,t] - y[h,t-1] for h in H for t in T2), name='defw1') 
m.addConstrs((w[h,t] >= y[h,t-1] - y[h,t] for h in H for t in T2), name='defw2') 

{(1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 21): <gurobi.Constr *Awaiting Mode

5. Contabilizar las paradas de la máquina.

In [29]:
# Definir z[t] en función de w[h,t]
m.addConstrs((w.sum('*', t) <= len(H)*z[t] for t in T2), name='defz') 

{2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>,
 22: <gurobi.Constr *Awaiting Model Update*>,
 23: <gurobi.Constr *Awaiting Model Update

Optimizar el modelo:

In [30]:
m.optimize()

Optimize a model with 25389 rows, 11094 columns and 72064 nonzeros
Variable types: 0 continuous, 11094 integer (11094 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 23599 rows and 495 columns
Presolve time: 0.11s
Presolved: 1790 rows, 10599 columns, 47970 nonzeros
Variable types: 0 continuous, 10599 integer (10599 binary)
Found heuristic solution: objective 66.0000000

Root relaxation: objective 0.000000e+00, 7968 iterations, 1.52 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0  888   66.00000    0.00000   100%     -    5s
H    0     0                      62.0000000    0.00000   100%     -    6s
H    0     0                      32.0000000    0.00000   100%     -   12s
     0     0    0.00000    0  917   32.000

  3798  3136    2.93683   81  807    9.00000    1.00000  88.9%   735  687s
  3865  3181    3.16410   86  747    9.00000    1.00000  88.9%   734  691s
  3925  3218    3.20064   89  728    9.00000    1.00000  88.9%   733  759s
  3985  3262    4.38889   96  532    9.00000    1.00000  88.9%   733  764s
  4105  3317    4.03274  107  549    9.00000    1.00000  88.9%   725  805s
  4216  3371    4.51241  112  562    9.00000    1.00000  88.9%   716  815s
  4295  3398    5.70769  125  506    9.00000    1.00000  88.9%   715  821s
  4361  3430    6.47421  137  440    9.00000    1.00000  88.9%   719  828s
  4478  3497    1.48837   30  832    9.00000    1.00000  88.9%   713  840s
  4615  3551    2.25000   42  607    9.00000    1.00000  88.9%   706  860s
  4732  3625    3.85714   47  541    9.00000    1.00000  88.9%   693  866s
  4884  3688    7.62500   80  373    9.00000    1.00000  88.9%   686  872s
  5029  3780    4.42636   53  552    9.00000    1.00000  88.9%   682  879s
  5252  3897    8.00000  

Mostrar la solución: instalaciones a construir, asignación de clientes a instalaciones y clientes no atendidos.

In [11]:
# Mostrar solucion
print('Orden de procesamiento de trabajos:')
print([j for t in T for j in J if x[j,t].x >= 0.9])

print('Numero de paradas: {}'.format(m.objVal))

print('Estado del carrete:')
for t in T:
    print('{}: {}'.format(t, [h for h in H if y[h,t].x >= 0.9]))

Orden de procesamiento de trabajos:
[5, 1, 7, 4, 2, 6, 3]
Numero de paradas: 3.0
Estado del carrete:
1: [1, 2, 4]
2: [2, 3, 5]
3: [2, 3, 5]
4: [1, 3, 4]
5: [1, 3, 4]
6: [1, 4, 5]
7: [1, 4, 5]


## Código completo

Reproducimos a continuación el código completo del ejemplo:

In [1]:
# Curso de implementación de programas lineales enteros
# Ejemplo: Modelo de job scheduling
# EPN (2019)

from gurobipy import *
try:
    # Conjuntos y parametros del modelo
    # Familia HH de conjuntos de hilos requeridos por cada trabajo
    HH = tupledict({1 : {2, 3, 5},
                    2 : {1, 3, 4},
                    3 : {1, 4, 5},
                    4 : {1, 3},
                    5 : {1, 2, 4},
                    6 : {1, 4, 5},
                    7 : {3, 5}})

    # Cantidad de hilos que pueden estar en el carrete
    B = 3

    # Conjunto de trabajos
    J = HH.keys()

    # Conjunto de turnos T y de turnos sin el primero
    T = J
    T2 = [i for i in T if i!=1]

    # Conjunto de todos los hilos, calculado como la union de Hj para j in J
    H = set()
    for j in J:
        H = H | HH[j]
    
    # Crear el objeto modelo
    m = Model('job-scheduling')

    # Definir variables
    # Asignacion turnos a trabajos
    x = m.addVars(J, T, vtype = GRB.BINARY, name="x")

    # Estado de hilos en el carrete
    y = m.addVars(H, T, vtype = GRB.BINARY, name="y")

    # Paradas de la maquina
    z = m.addVars(T2, vtype = GRB.BINARY, name="z")

    # Auxiliar: cambio de los hilos en el carrete
    w = m.addVars(H, T2, vtype = GRB.BINARY, name="z")
    
    # Funcion objetivo: minimizar paradas de la maquina
    m.setObjective(z.sum('*'), GRB.MINIMIZE)

    # Restricciones
    # A cada turno se le asigna exactamente un trabajo
    m.addConstrs((x.sum('*', t) == 1 for t in T), name='turno_t') 

    # A cada trabajo se le asigna exactamente un turno
    m.addConstrs((x.sum(j, '*') == 1 for j in J), name='turno_j') 
    
    # Respetar capacidad del carrete
    m.addConstrs((y.sum('*', t) <= B for t in T), name='carrete') 
    
    # Cargar todos los hilos requeridos para procesar un trabajo
    m.addConstrs((x[j,t] <= y[h,t] for t in T for j in J for h in HH[j]), name='hilos') 
    
    # Definir w[h,t] en función de y[h,t] y y[h,t-1]
    m.addConstrs((w[h,t] >= y[h,t] - y[h,t-1] for h in H for t in T2), name='defw1') 
    m.addConstrs((w[h,t] >= y[h,t-1] - y[h,t] for h in H for t in T2), name='defw2') 
    
    # Definir z[t] en función de w[h,t]
    m.addConstrs((w.sum('*', t) <= len(H)*z[t] for t in T2), name='defz')     

    # Resolver el modelo
    m.optimize()
    
    # Mostrar solucion
    print('Orden de procesamiento de trabajos:')
    print([j for t in T for j in J if x[j,t].x >= 0.9])

    print('Numero de paradas: {}'.format(m.objVal))

    print('Estado del carrete:')
    for t in T:
        print('{}: {}'.format(t, [h for h in H if y[h,t].x >= 0.9]))
        
except GurobiError as e:
    print('Se produjo un error de Gurobi: codigo: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error deatributo')

Optimize a model with 220 rows, 120 columns and 615 nonzeros
Variable types: 0 continuous, 120 integer (120 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 6.0000000
Presolve removed 104 rows and 30 columns
Presolve time: 0.00s
Presolved: 116 rows, 90 columns, 481 nonzeros
Variable types: 0 continuous, 90 integer (90 binary)

Root relaxation: objective 0.000000e+00, 143 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   68    6.00000    0.00000   100%     -    0s
H    0     0                       5.0000000    0.00000   100%     -    0s
H    0     0                       4.0000000    0.00000   100%     -    0s
H    0     0                       3.0000000    0.00000   100%     - 

In [15]:
import random
help(random.randint)

Return random integer in range [a, b], including both end points.
        


In [21]:
random.randint(1,3)

3